# Data preparation

## Creating purchases columns in user_id - SQL method

**Creation de la colonne purchases**

On créé une colonne "purchases" pour spécifier si l'utilisateur à déjà acheté au moins un produit.  
On le fait via la console GCP avec les commande SQL ci dessous.  

Dans un premier temps on doit créé la colonne "purchases" de type booléen qui sera vide.  
Puis on initialise toutes les valeurs à False avec la requête ci dessous.

```   
UPDATE `marbotic.marbotic_dataset.user_id_f`  
SET purchases = False  
WHERE TRUE  
```


On mets à jour les profils utilisateurs ayant effectué au moins un achat avec la requête ci dessous.  

```
UPDATE `marbotic.marbotic_dataset.user_id_f`   
SET purchases = True  
WHERE Purchases_PY1Y = 1  
OR  
Purchases_PLY = 1  
OR   
Purchases_MEGR = 1  
OR  
Purchases_PLM = 1  
OR   
Purchases_ME1 = 1  
OR   
Purchases_EY3M = 1  
OR  
Purchases_tier_upgrade_0_to_1 = 1  
OR  
Purchases_PM3M = 1  
OR  
Purchases_MPGR = 1  
OR  
Purchases_EY1Y = 1  
OR  
Purchases_PY3M = 1  
```

Enfin on ajoute cette colonne à la table users_metrics 

**Vérifier s'il faut créer la colonne à la main avant**

```
UPDATE `marbotic.marbotic_dataset.users_metrics_f` as um
SET um.purchases = tmp.purchases
FROM (SELECT um.user_id, MAX(ui.purchases) as purchases
FROM `marbotic.marbotic_dataset.users_metrics_f` as um LEFT JOIN `marbotic.marbotic_dataset.user_id_f` as ui
ON um.user_id = ui.User_id
GROUP BY um.user_id
) as tmp
WHERE um.user_id = tmp.user_id
```


## Creating game_events and setup_events in session_metrics - SQL method

**Creation de la colonne game_events**

On créé une colonne "game_events" pour compter le nombre d'event de jeu par session.  
On le fait via la console GCP avec les commande SQL ci dessous.  

Dans un premier temps on doit créé la colonne "game_events" de type Float qui sera vide.  
Puis on la complète en sommant les events de jeu

```
UPDATE `marbotic.marbotic_dataset.sessions_metrics_f` 
SET game_events = 
 event_type_Activation_Game + event_type_Game_Stop + event_type_Scaffolding_Reset + event_type_Game_Pause + event_type_Game_Start + 
 event_type_Error_Error + event_type_Scene_Enter + event_type_Scene_Leave + event_type_Action_Click + event_type_Activity_Stop + 
 event_type_Activity_Start + event_type_Activity_Good_Answer + event_type_Activity_Wrong_Answer + event_type_Scaffolding_Scaffolding
WHERE True
```

**Creation de la colonne setup_event**

De la même façon on crée la colonne setup_event pour compter le nombre d'event de setup par session.  
On le fait via la console GCP avec les commande SQL ci dessous.  

Dans un premier temps on doit créé la colonne "setup_events" de type Float qui sera vide.  
Puis on la complète en sommant les events de setup

```
UPDATE `marbotic.marbotic_dataset.sessions_metrics` 
SET setup_events = 
event_type_Profile_Create + 
event_type_Profile_Update +
event_type_Account_Sign_In +
event_type_OB_ProfileName +
event_type_OB_ProfileBirthdate +
event_type_OB_KeyRedeem +
event_type_OB_KeyFail +
event_type_OB_RenewalSelection +
event_type_Account_Sign_Up +
event_type_OB_ProductSelection +
event_type_OB_ProductSK +
event_type_OB_ProductHelp +
event_type_OB_ProductActivation +
event_type_OB_RenewalSuccess+
event_type_OB_ProductSeeMore+
event_type_OB_ProductOther+
event_type_Account_Sign_Out+
event_type_OB_CommPreview+
event_type_OB_DoYouHaveKit+
event_type_OB_HasKitYes+
event_type_OB_Email+
event_type_OB_VerifCode+
event_type_Activation_New_Activation+
event_type_Activation_Product+
event_type_Activation_Piece+
event_type_OB_Success+
event_type_OB_RenewalFail+
event_type_OB_ClickTrial+
event_type_Popup_Open+
event_type_Popup_Close+
event_type_OB_HasKitNo+
event_type_OB_DiscoverTrial+
event_type_Account_Update_Request+
event_type_Account_Update_Success+
event_type_Profile_Delete+
event_type_OB_TrialStart+
event_type_OB_ClickStore+
event_type_OB_HasKeyNo+
event_type_OB_Type+
event_type_OB_KeySuccess+
event_type_Profile_Reset_Settings_and_Progression+
event_type_Action_ConversionClickRedeem+
event_type_Action_ConversionClickSubscribe+
event_type_Toast_Appear+
event_type_Toast_Disappear
WHERE True
```

## Imports

In [25]:
import pandas as pd
import json
import numpy as np
from datetime import datetime,timedelta


#Imports Bigquery
from google.cloud import bigquery
from google.oauth2 import service_account


## GCP config

In [2]:
#Localisation du projet name et table_id sur gcp
#TODO : Remplacer le nom du projet et du dataset sur GCP 
project="marbotic"
dataset = "marbotic_dataset"

In [3]:
#intégration des credentials
#TODO : Remplacer le path de credential d'accès à notre projet GCP
key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],)

## Creating table session_augmented

In [4]:
def extract(project,credentials):
    '''Extracting first data and creating feature : unique game per session, Time_spent sum, good_answer_ratio'''
    
    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT
    COUNT(DISTINCT ep.Activity_Name) as unique_activity, 
    sm.session_id,sum(Time_Spent) as Time_Spent,
    any_value(Action_Element_Name) as Action_Element_Name,
    any_value(Profile_Age_days) as Profile_Age_days,
    (SAFE_DIVIDE(SUM(event_type_Activity_Good_Answer) , (SUM(event_type_Activity_Good_Answer)
    + SUM(event_type_Activity_Wrong_Answer))))*100 as good_answer_ratio
    FROM `marbotic.marbotic_dataset.event_properties` as ep
    JOIN `marbotic.marbotic_dataset.sessions_metrics` as sm on ep.session_id = sm.session_id
    GROUP BY sm.session_id
    ORDER BY unique_activity DESC
    """

    query_job = client.query(query) 
    results=query_job.result().to_dataframe()

    return pd.DataFrame(results)

In [5]:
def extract_session_metrics(project,credentials):
    '''Extracting information from session metrics table'''

    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT *
    FROM `marbotic.marbotic_dataset.sessions_metrics`
    """

    query_job = client.query(query)  
    results=query_job.result().to_dataframe() 

    return pd.DataFrame(results)

In [6]:
def extract_user_metrics(project,credentials):
    '''Extracting information from user metrics'''
    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT user_creation_time, user_id
    FROM `marbotic.marbotic_dataset.users_metrics`
    """

    query_job = client.query(query)  
    results=query_job.result().to_dataframe() 

    return pd.DataFrame(results)

In [7]:
def extract_user_type(project,credentials):
    '''Extracting the type for each user'''
    client = bigquery.Client(project=project, credentials=credentials)

    query = """
    SELECT any_value(Type) as Type, user_id
    FROM `marbotic.marbotic_dataset.user_properties`
    GROUP BY user_id
    """

    query_job = client.query(query)  
    results=query_job.result().to_dataframe() 

    return pd.DataFrame(results)

In [8]:
#On charge toutes les tables dont on a besoin
first_data = extract(project,credentials)
session_metrics_df = extract_session_metrics(project,credentials)
user_metrics_df = extract_user_metrics(project,credentials)
user_type_df = extract_user_type(project,credentials)

In [9]:
#on merge les première colonnes crées avec la table sessions metrics
sm_df = session_metrics_df.merge(first_data, on='session_id')

In [10]:
#On ajoute une colonne de la durée de chaque session
sm_df['duration_min'] = (sm_df['session_end']-sm_df['session_start']).apply(lambda x: x.total_seconds()/60)

In [11]:
#on merge la date de création dans le cas ou on veut calculer la maturité du user au moment de la session.
sm_df = sm_df.merge(user_metrics_df, on='user_id')

In [12]:
def nb_session(user_id,session_start):
    '''Compute number of session before current session'''
    return sm_df[(sm_df['user_id']==user_id) & (sm_df['session_start']<session_start)].shape[0]

In [13]:
sm_df['previous_session'] = sm_df.apply(lambda x:nb_session(x.user_id,x.session_start), axis=1)

In [17]:
def last_session_time(user_id,session_start):
    '''Compute time since last session'''
    last_session = sm_df[(sm_df['user_id']==user_id) & (sm_df['session_start']<session_start)]\
    .sort_values(by='session_start',ascending=True)\
    .groupby('user_id').last()['session_start']
    if len(last_session.values)==0:
        return timedelta(0)
    else:
        last_session = last_session.dt.to_pydatetime()[0].replace(tzinfo=None)
        return (session_start.to_pydatetime().replace(tzinfo=None) - last_session)

In [18]:
sm_df['time_since_last_session'] = sm_df.apply(lambda x:last_session_time(x.user_id,x.session_start), axis=1)

In [19]:
#Conversion de la colonne time_since_last_session timedelta en timestampe[min]
sm_df['time_since_last_session_min']=sm_df['time_since_last_session'].map(lambda x : x.total_seconds()/60)

In [20]:
#Ajout de la maturité au moment du début de la sessions
sm_df["week_maturity"] =(sm_df["session_start"]-sm_df["user_creation_time"]).map(lambda x: int(np.ceil(x.days/7))+1)

In [21]:
sm_df = sm_df.merge(user_type_df, on='user_id')

In [22]:
#Creating columns with day & month of the session
sm_df['session_start_day'] = sm_df['session_start'].apply(lambda x: x.weekday()+1)
sm_df['session_start_month'] = sm_df['session_start'].apply(lambda x: x.month)

In [23]:
#On drop la colone time_since_last_session en timedelta car incompatbile BQ
sm_df.drop('time_since_last_session',axis=1,inplace=True)

## Labelling

In [26]:
#On considère le seuil de churn à 3 semaine.
#On est en situation de churn si la session suivante arrive dans plus de 3 semaines.
CHURN_THRESHOLD = timedelta(days=21)

#On utilise une constante TODAY pour evaluer le risque de churn des dernières session.
#Dans notre cas TODAY est le max de session end. On considère que l'extract s'arrète au moment de la dernière session de jeu dont on dispose
TODAY = datetime(2022, 5, 13, 0, 0)

In [27]:
def labelling_churn(user_id,session_end,session_start,CHURN_THRESHOLD,today):
    '''Compute time to next session, if superior to threshold churn_risk = 1'''
    next_session = sm_df[(sm_df['user_id']==user_id) & (session_end<sm_df['session_end'])]\
    .sort_values(by='session_end',ascending=True)\
    .groupby('user_id').first()['session_start']
    if len(next_session.values) == 0:
        if today - session_start.to_pydatetime().replace(tzinfo=None)> CHURN_THRESHOLD :
            return 1
        else:
            return 0
    else:
        delta = (next_session - session_end)
        if delta.iloc[0] < CHURN_THRESHOLD:
            return 0
        else:
            return 1

In [28]:
sm_df['churn_risk'] = sm_df.apply(lambda x:labelling_churn(x.user_id,x.session_end,x.session_start,CHURN_THRESHOLD,TODAY), axis=1)

## Upload in BQ

In [31]:
def upload_session_augmented(df,credentials, project,dataset,table):
    '''Uploading sessions_augmented on GCP'''
    table_id = f"{dataset}.{table}"
    client = bigquery.Client(project,credentials)

    job = client.load_table_from_dataframe(df, table_id) 

    job.result()  
    table = client.get_table(table_id) 
    print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))

In [34]:
#TODO : Uncomment this line if we need to upload again sessions augmented table
#upload_session_augmented(sm_df,credentials, project,dataset,"sessions_augmented_f")

Loaded 18816 rows and 87 columns to marbotic_dataset.sessions_augmented_f
